### 根据用户打分的cb算法

In [1]:
import os
import operator
import sys
import Ipynb_importer
from utils import read

importing Jupyter notebook from F:\FictionRecommend\recall\CB\ratings_cb\utils\read.ipynb


In [2]:
movies_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\movies.csv"
ratings_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\ratings.csv"

In [8]:
def get_up(item_cate, input_file):
    '''
    用户对电影分类的偏好
    '''
    if not os.path.exists(input_file):
        return {}
    fp = open(input_file, "r", encoding="utf-8")
    line_num = 0
    # 评分阈值
    score_thr = 4.0
    
    # user_id对应每一个类别的得分
    record = {}
    up = {}
    
    for line in fp:
        if line_num == 0:
            line_num += 1
            continue
        
        item = line.strip().split(",")
        if len(item) < 4:
            continue
        
        user_id, item_id, rating, timestamp = item[0], item[1], float(item[2]), int(item[3])
        
        # 将打分低的item过滤掉
        if rating < score_thr:
            continue
            
        # 过滤掉item不在类别list中的
        if item_id not in item_cate:
            continue
        
        # 作时间衰减得到时间打分
        time_score = get_time_score(timestamp)
        if user_id not in record:
            record[user_id] = {}
        
        for fix_cate in item_cate[item_id]:
            if fix_cate not in record[user_id]:
                # 先初始化为0
                record[user_id][fix_cate] = 0
            
            # 计算类别得分(用户对当前item的评分 * 时间得分 * 当前类别在当前item下的权重得分)[可优化]
            record[user_id][fix_cate] += rating * time_score * item_cate[item_id][fix_cate]
            
    fp.close()
    
    # return record
    
    topn = 2
    for user_id in record:
        if user_id not in up:
            up[user_id] = []
        
        # 对用户喜欢类别的得分进行归一化
        total_score = 0
        # 按照得分排序，取topn个最受欢迎的类别
        for item in sorted(record[user_id].items(), key=operator.itemgetter(1), reverse=True)[:topn]:
            up[user_id].append((item[0], item[1]))
            total_score += item[1]
            
        for index in range(len(up[user_id])):
            # 除sum归一化
            up[user_id][index] = (up[user_id][index][0], round(up[user_id][index][1] / total_score, 3))
        
    return up

In [4]:
def get_time_score(timestamp):
    '''
    因为数据集的时间比较久远，所以采用在数据集中选择最大的时间来做时间衰减
    '''
    # 最大时间戳
    fix_time_stamp = 1537799250
    # 转为天
    total_ser = 24 * 60 * 60
    # 计算和参数时间戳的差，并转为天数
    delta = (fix_time_stamp - timestamp) / total_ser / 100
    
    # 差距越小得分越高，最高分1分
    return round(1 / (1 + delta), 3)

In [10]:
def recom(cate_item_sort, up, user_id, topn=20):
    '''cb召回'''
    # 如果用户没在画像中则返回空
    if user_id not in up:
        return {}
    
    recom_result = {}
    if user_id not in recom_result:
        recom_result[user_id] = []
    
    # 对每一个类别分别进行召回
    for item in up[user_id]:
        # 类别
        cate = item[0]
        # 权重
        ratio = item[1]
        
        # 各类别召回的个数 = topn * 类别权重
        # 避免出现0
        num = int(topn * ratio) + 1
        if cate not in cate_item_sort:
            continue
        recom_list = cate_item_sort[cate][:num]
        
        # 添加推荐解释(很重要)[看看item是从哪个类别中召回的]
        recom_list = [e + "_" + cate for e in recom_list]
        recom_result[user_id] += recom_list
    
    return recom_result

In [ ]:
def run_main():
    # 平均得分
    avg_score = read.get_avg_score(ratings_file)
    # item所属的类别，类别到item的倒排
    item_cate, cate_item_sort = read.get_item_cate(avg_score, movies_file)
    
    # 用户画像
    up = get_up(item_cate, ratings_file)
    
    print(recom(cate_item_sort, up, "1"))

In [5]:
# 平均得分
avg_score = read.get_avg_score(ratings_file)
# item所属的类别，类别到item的倒排
item_cate, cate_item_sort = read.get_item_cate(avg_score, movies_file)

In [7]:
record = get_up(item_cate, ratings_file)
record

{'1': {'Action': 1.6742100000000006,
  'Adventure': 1.5870750000000002,
  'Animation': 0.51258,
  'Children': 0.7039650000000001,
  'Comedy': 2.0838300000000003,
  'Crime': 0.8950049999999999,
  'Drama': 1.6726950000000005,
  'Fantasy': 0.8175749999999999,
  'Film-Noir': 0.01875,
  'Horror': 0.323685,
  'Musical': 0.41856,
  'Mystery': 0.29309999999999997,
  'Romance': 0.54603,
  'Sci-Fi': 0.69606,
  'Thriller': 0.956475,
  'War': 0.502245,
  'Western': 0.15243},
 '10': {'Action': 1.47573,
  'Adventure': 1.7372649999999996,
  'Animation': 0.8034149999999999,
  'Children': 0.7320224999999999,
  'Comedy': 4.626785,
  'Crime': 0.660725,
  'Drama': 4.312620000000001,
  'Fantasy': 0.8512000000000001,
  'IMAX': 0.69255,
  'Musical': 0.32827249999999997,
  'Romance': 4.0319899999999995,
  'Thriller': 0.4788,
  'War': 0.11875},
 '100': {'Action': 0.544901,
  'Adventure': 0.2971885,
  'Animation': 0.019,
  'Children': 0.07885,
  'Comedy': 2.0216380000000003,
  'Crime': 0.33212949999999997,
  'D

In [9]:
up = get_up(item_cate, ratings_file)
up

{'1': [('Comedy', 0.554), ('Action', 0.446)],
 '10': [('Comedy', 0.518), ('Drama', 0.482)],
 '100': [('Drama', 0.519), ('Comedy', 0.481)],
 '101': [('Drama', 0.632), ('Comedy', 0.368)],
 '102': [('Comedy', 0.549), ('Thriller', 0.451)],
 '103': [('Drama', 0.656), ('Comedy', 0.344)],
 '104': [('Comedy', 0.634), ('Romance', 0.366)],
 '105': [('Comedy', 0.51), ('Drama', 0.49)],
 '106': [('Adventure', 0.529), ('Fantasy', 0.471)],
 '107': [('Drama', 0.645), ('Comedy', 0.355)],
 '108': [('Drama', 0.611), ('Romance', 0.389)],
 '109': [('Drama', 0.695), ('Crime', 0.305)],
 '11': [('Drama', 0.531), ('Action', 0.469)],
 '110': [('Drama', 0.646), ('Comedy', 0.354)],
 '111': [('Comedy', 0.835), ('Drama', 0.165)],
 '112': [('Drama', 0.577), ('Action', 0.423)],
 '113': [('Drama', 0.605), ('Romance', 0.395)],
 '114': [('Adventure', 0.554), ('Fantasy', 0.446)],
 '115': [('Comedy', 0.576), ('Action', 0.424)],
 '116': [('Action', 0.518), ('Romance', 0.482)],
 '117': [('Drama', 0.597), ('Comedy', 0.403)],

In [21]:
# 召回
recom(cate_item_sort, up, "1")

{'1': ['467_Comedy',
  '496_Comedy',
  '626_Comedy',
  '633_Comedy',
  '876_Comedy',
  '1151_Comedy',
  '3086_Comedy',
  '3096_Comedy',
  '3473_Comedy',
  '3531_Comedy',
  '3567_Comedy',
  '3851_Comedy',
  '876_Action',
  '1631_Action',
  '2196_Action',
  '4180_Action',
  '5244_Action',
  '5490_Action',
  '5746_Action',
  '6835_Action',
  '26169_Action']}